# Customize a flavor

Sometimes you need very specific dependencies or versions of dependencies in the Exasol UDFs. In such case you can customize a Script-Language Container.

## Setup
### Open Secure Configuration Storage

In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

### Instantiate SLCT Manager

We create an instance of the SlctManager class from the notebook connector. SLCT Manager, which stands for "Script-Languages-Container-Tools" Manager. 
This class has some utility function which simplifies the use of the exaslct API.

In [ ]:
from exasol.nb_connector import slct_manager
slctmanager = slct_manager.SlctManager(ai_lab_config)

### Import some utility functions

In [ ]:
%run ./utils/file_system_ui.ipynb

## Customize

### Flavor Definition
The following diagram shows a high level overview of the build steps for a script languages container.


![image.png](slc_main_build_steps.svg)



For customizing a flavor usually the `flavor_customization` build step is most important. It contains everything you need to add dependencies. The remaining build steps should be only changed with care, but sometimes some dependencies are defined in other build steps because the script client depends on them. 
Check the directory structure of the selected flavor:


In [ ]:
show_directory_content(slctmanager.slc_dir.flavor_dir, 2)

### Flavor Customization Build Step

The **flavor_customization** build step consists of a Dockerfile and several package lists which can be modified. We recommend to use the package lists to add new packages to the flavor and only modify the Dockerfile if you need very specific changes, like adding additional resources.

In [ ]:
show_directory_content(slctmanager.slc_dir.flavor_dir / "flavor_customization")

The Dockerfile consists of two parts. The first part installs the packages from the package list and should only be change with care. The second part is free for your changes. Read the description in the Dockerfile carefully to find out what you can and shouldn't do.

#### Package Lists
The package lists have a unified format. Each line consists of the package name and the package version separated by "|", e.g `xgboost|1.3.3`. You can comment out a whole line by adding"#" at the beginning. You can also add a trailing comment to a package definition by adding a "#" after the package definition. We usually recommend to pin the version, such that there are no surprises for which version gets installed.

In [ ]:
show_files([slctmanager.custom_pip_file])

We are now going to append the "xgboost" Python package to one of the package lists by adding `xgboost|2.0.3` and `scikit-learn|1.5.0` to the `flavor_customization/packages/python3_pip_packages` file. 
Notes:
 - running the following command multiple times will iteratively append the packages
 - you can also click on the link and modify the file directly

In [ ]:
xgboost_pkg = slct_manager.PipPackageDefinition(pkg="xgboost", version="2.0.3")
scikit_learn_pkg = slct_manager.PipPackageDefinition(pkg="scikit-learn", version="1.5.0")
slctmanager.append_custom_packages([xgboost_pkg, scikit_learn_pkg])
show_files([slctmanager.custom_pip_file])

#### Rebuilding the customized Flavor

After changing the flavor you need to rebuild it. You can do it by running `export` again. Exaslct automatically recognizes that the flavor has changed and builds a new version of the container.

In [ ]:
slctmanager.export()

Note: Your old container doesn't get lost, because when you change a flavor your container gets a new hash code. If you revert your changes the system automatically uses the existing cached container. Below you can see the content of the cache directory for the containers.

In [ ]:
show_directory_content(slctmanager.working_path.output_path / "cache" / "exports")

#### Upload the Container to the Database
To use our container we need to upload it to the BucketFS . If the build machine has access to the BucketFS we can do it with the exaslct upload command, otherwise you need to export the container and transfer it to a machine that has access to the BucketFS and upload it via curl, as described in our documentation.

With the following command we upload the new script language container.


In [ ]:
slctmanager.upload()

This command also stores the activation statement in the ai-lab-config. You can verify it with:

In [ ]:
ai_lab_config.get(slct_manager.ACTIVATION_KEY)

The syntax is: `alias=url`. The alias is used when defining an UDF, the URL only for the language activation.
For simplicity the `SlctManager` provides access to the alias, as you need it later to define the UDF's:

In [ ]:
slctmanager.language_alias

You can now continue [testing the uploaded container](./test_slc.ipynb).